# Modeling the evolution of an etch recipe in Deep silicon etching

To Do for next version:
Cellular automata implementation of dry silicon etching.
Cells in a 3D grid are intialized with a state value of 1. As etching occurs, the cell state is subtracted from at a user-defined rate that, for isotropic steps, its adjusted by the calculated angle between the surface normals and the center vertical axis of the etched feature.

Simplify Etch conditions and simulation parameters

Verify recipe steps for steps that are  Bosch or isotropic etching, vert_rate, horiz_rate, and bosch_vert_step. 

add image of a mask superimposed to the simulation

In the following Notebook we will use the mask below but feel free to change it or upload your own.

<img align="center" src="img/fillet_sq_example_mask.png" width="300" />



During this notebook you will learn to  predict and model etch profiles from dry silicon etching based on known etch rates. You will be able to change multiple parameters, including the etch rate and exposure time.

To model continuous etching into silicon wafer on a Deep Silicon etching tool you will need to specify the correct parameters. In this case, you will simulate dry etching in a gas plasma. The simulation takes a mask image, which is a binary image that represents the layout of the pattern to be etched, and generates an output contour that simulates the etching of the mask into the silicon wafer. The code then creates various visualizations of the simulated etching process, including a 3D surface plot and a contour plot. You can see the resulting plots in the directory '/img/'.

In [14]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
from scipy.spatial import distance as dist
from matplotlib.path import Path
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
from shapely.ops import cascaded_union
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import splprep, splev
mpl.style.use('default')

"""
Model continuous etching into silicon wafer with
based known etch rates.
"""   

def horiz_etch(cont,horiz_rate,t_step,norm_span,sm_window):
    out_cont = np.zeros_like(cont)
    for p, point in enumerate(cont):
        # calculate normal to point
        # enable looping index to beginning
        if p + norm_span > len(cont)-1:
            dummy_p = 0
        else:
            dummy_p = p
        p_1 = cont[dummy_p-norm_span]
        p_2 = cont[dummy_p+norm_span]
        tan_vec = np.array([[p_2[0]-p_1[0]],
                         [p_2[1]-p_1[1]]])
        norm_vec = np.matmul(rot90_mat,tan_vec)
        unit_norm = norm_vec/np.linalg.norm(norm_vec)
        new_pt = point + horiz_rate*t_step*np.reshape(unit_norm,(1,2))
        out_cont[p,:] = new_pt
    
    out_cont[-1,0] = out_cont[0,0]
    out_cont[-1,1] = out_cont[0,1]
    tck, u = splprep(out_cont.T, u=None, s=0, per=1) 
    u_new = np.linspace(u.min(), u.max(), len(cont))
    x_spline, y_spline = splev(u_new, tck, der=0)
    out_cont = np.hstack((np.reshape(np.array(x_spline),[len(x_spline),1]),
                          np.reshape(np.array(y_spline),[len(y_spline),1])))        
    
    return out_cont


The function horiz_etch applies a normal step to a contour by calculating the normal vector at each point of the contour and moving each point along the normal vector by a given distance. This function also smooths the resulting contour using a spline function.

Now we set several parameters for the simulation, including the etch rates of two gases (C4F8 and SF6), the bias voltage, the etching time, and the opening size. Load a binary mask image and apply a Gaussian filter to the image using OpenCV's cv2.GaussianBlur function. We will need to find the contours in the image using OpenCV's cv2.findContours function and draw the contours onto the image using Matplotlib's cv2.drawContours function. 



In [ ]:
    
C4F8 = 100  # sccm cubic centimeters per minute Octafluorocyclobutane 
SF6 = 300  # sccm
bias = 10  # volts
time = 600  # seconds
opening = 100  # um

plt.close('all')

### Load mask here
im_dir = '/img/'
im_file = 'fillet_sq_example_mask.png' 
im_path = 'img/fillet_sq_example_mask.png' #### SELECT OR UPLOAD THE MASK HERE
curr_im = cv2.imread(im_path, cv2.IMREAD_ANYDEPTH)   
curr_im = cv2.GaussianBlur(curr_im,(3,3),0)



Next, we need to set several more parameters for the simulation, including the start and end times, the time step, the height and width of the image, the number of points to simulate, the contour reading step, the normal span, the smoothing window length, the vertical etch rate, the horizontal etch rate, the pixel-to-micron conversion factor, the colormap to use for visualizations, and the expected range of depth for the color bar.

We then generate an array of x-axis values using NumPy's np.linspace function and calculate the corresponding y-axis values using the contour from the mask image. The code then generates a meshgrid using NumPy's np.meshgrid function and applies the horiz_etch function to the contour at each time step to simulate the etching process. The resulting 3D surface is plotted using Matplotlib's Axes3D.plot_surface function.

In [ ]:

rgb_im = cv2.cvtColor(curr_im, cv2.COLOR_GRAY2RGB)
conts, hier = cv2.findContours(curr_im, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)    
conts_im = cv2.drawContours(rgb_im, conts, -1, (0,255,0),3)
dummy_i = im_file.find('.png')
out_file = im_dir + im_file[:dummy_i] + '_out' + im_file[dummy_i:]
cv2.imwrite(out_file, conts_im)

######CHANGE TIME STEP HERE
t_start = 0
t_end = 600  # seconds
t_step = 5
h = curr_im.shape[0]
w = curr_im.shape[1]
n_points = 600
contour_read_step = 5
topo_im = np.zeros_like(curr_im)
norm_span = 3
window_len = 17
rot90_mat = np.array([[np.cos(np.pi/2), -np.sin(np.pi/2)],
                      [np.sin(np.pi/2), np.cos(np.pi/2)]])
vert_rate = 287/600  # um/s
horiz_rate = 77/600  # um/s
pixel_um_conv = 251/90.4672  # px/um
cmap = 'gnuplot'  
vmin = -290  # expected range of depth for color bar (min)
vmax = 0
rstride = 2
cstride = 2

In [ ]:

x_axis = np.linspace(0,w/pixel_um_conv,n_points)
y_axis = np.linspace(0,h/pixel_um_conv,n_points)
xv,yv = np.meshgrid(x_axis,y_axis)
x_points = np.ravel(xv)
x_points = x_points.reshape((len(x_points),1))
y_points = np.ravel(yv)
y_points = y_points.reshape((len(y_points),1))
grid_point_pairs = np.hstack((x_points,y_points))

conts_paths = {}
conts_polys = {}
topo_data = {}
unit_norm_vectors = {}

for c, cont in enumerate(conts):
    x = []
    y = []
    for p, point in enumerate(cont):
        if p%contour_read_step == 0:
            x.append(point[0][0]/pixel_um_conv)
            y.append(point[0][1]/pixel_um_conv)    
    x[-1] = x[0]
    y[-1] = y[0]
    points = np.hstack((np.reshape(np.array(x),[len(x),1]),
                       np.reshape(np.array(y),[len(y),1])))        

    tck, u = splprep(points.T, u=None, s=0.0, per=1) 
    u_new = np.linspace(u.min(), u.max(), len(cont))
    x_spline, y_spline = splev(u_new, tck, der=0)

    points = np.hstack((np.reshape(np.array(x_spline),[len(x_spline),1]),
                                 np.reshape(np.array(y_spline),[len(y_spline),1])))


    temp_poly = Polygon(points)
    temp_path = Path(temp_poly.exterior,closed=True)    
    conts_paths[c] = temp_path  
    conts_polys[c] = temp_poly
    
    unit_norm_vectors[c] = np.zeros_like(conts_paths[c].vertices)
    topo_data[c] = np.zeros((grid_point_pairs.shape[0],1))  

    
x = grid_point_pairs[:,0].reshape(xv.shape)
y = grid_point_pairs[:,1].reshape(yv.shape)

fig2, ax2 = plt.subplots(figsize=(8,7))

dummy_cont_count = len(conts_paths)

topo = []


Dry etching is a process used in semiconductor manufacturing to selectively remove layers of material from a substrate. It involves the use of plasma, which is a partially ionized gas, to react with the material and etch it away.

The simulation in the "solve the etching of the mask" section is a simplified representation of dry etching. In this simulation, the mask represents a protective layer on top of the substrate, and the etchant represents the plasma used in dry etching.

The simulation works by using a set of rules to determine how the etchant reacts with the mask and substrate. When the etchant encounters the mask, it bounces off without reacting. However, when it encounters the substrate, it reacts and etches away a portion of the material.

By iterating through this process multiple times, the simulation simulates the dry etching process and shows how the substrate is etched away in a pattern determined by the mask.

In [ ]:

##### solve the etching of the mask
for i_t,t in enumerate(range(t_start, t_end, t_step)):

    print('solving time: ', t)
    z_mask = np.zeros_like(xv)  
    topo.append(np.zeros_like(xv))
    cont_loop = True
    overlap = False
    cummul_paths = {}
    c = 0
    
    while cont_loop == True:
        print('  checking for OVERLAP with contour: ', c)
        other_conts = list(range(dummy_cont_count))
        other_conts.pop(other_conts.index(c))
        for oc in other_conts:
            for pt in conts_paths[oc].vertices:
                if conts_paths[c].contains_point(pt):
                    overlap = True
                    break
            if overlap == True: break
        
        if overlap == True: 
            print('     overlap detected')
            polys = [conts_polys[poly] for poly in list(conts_polys.keys())]
            new_cont = cascaded_union([poly if poly.is_valid 
                                       else poly.buffer(0) for poly in polys])
            try:
                x_temp,y_temp = new_cont.exterior.xy
                x_temp[-1] = x_temp[0]
                y_temp[-1] = y_temp[0]
                temp_cont = np.hstack((np.reshape(np.array(x_temp),[len(x_temp),1]),
                                       np.reshape(np.array(y_temp),[len(y_temp),1])))
                tck, u = splprep(temp_cont.T, u=None, s=13, per=1) 
                u_new = np.linspace(u.min(), u.max(), len(cont))
                x_spline, y_spline = splev(u_new, tck, der=0)
                points = np.hstack((np.reshape(np.array(x_temp),[len(x_temp),1]),
                                    np.reshape(np.array(y_temp),[len(y_temp),1])))
                cummul_paths[c] = Path(points,closed=True)
                cont_loop = False
            except:
                overlap = False       
        if overlap == False:
            cummul_paths[c] = conts_paths[c]
            c += 1
            if c == dummy_cont_count: cont_loop = False
    for c in conts_paths:
        print('  solving VERT etch in contour: ', c)
        new_cont_points = horiz_etch(conts_paths[c].vertices,horiz_rate,
                                     t_step,norm_span,window_len)

        conts_paths[c] = Path(new_cont_points,closed=True)
        conts_polys[c] = Polygon(new_cont_points)

        inside = conts_paths[c].contains_points(grid_point_pairs)
        z_mask += inside.astype(int).reshape(xv.shape)
    z_mask[z_mask>0] = 1
    z_step = z_mask * (vert_rate*t_step)
    try:
        topo[i_t] =  topo[i_t-1] - z_step
    except:
        topo[i_t] -= z_step
        
    ax2.plot(t,vert_rate/horiz_rate,'k')
        
    fig_2d_cont, ax1_2d_cont = plt.subplots(figsize=(13,12))
    for c in cummul_paths:
        print('  solving HORIZ etch in cummulative contour: ', c)
            
        curr_path = cummul_paths[c]        
        try:
            updated_cont = horiz_etch(curr_path.vertices,horiz_rate,
                                      t_step,norm_span,window_len)
        except:
            pass
        dummy_cont_path = Path(updated_cont,closed=True)
        patch = patches.PathPatch(dummy_cont_path, fill=False, lw=2)
        ax1_2d_cont.add_patch(patch)   
        ax1_2d_cont.plot(dummy_cont_path.vertices[:,0],
                 dummy_cont_path.vertices[:,1],'k')
    
    contourplot = plt.contourf(x, y, topo[i_t], 500, cmap=cmap,vmin=vmin, vmax=vmax)
    title_str = 't = %s s' % str(t)
    plt.title(title_str)
    ax1_2d_cont, _ = mpl.colorbar.make_axes(plt.gca())
    cbar = mpl.colorbar.ColorbarBase(ax1_2d_cont, cmap=None,
                                     norm=mpl.colors.Normalize(vmin=vmin, vmax=vmax),
                                     label=' etch depth [um]')  
    ax1_2d_cont.autoscale()    
    out_fig = 'img/' + \
    str(t) + '.png'
    plt.savefig(out_fig, bbox_inches='tight')
    plt.close()

    fig_3d_surf = plt.figure(figsize=(24,10))
    ax2_3d_surf = fig_3d_surf.add_subplot(111, projection='3d')

    surf = ax2_3d_surf.plot_surface(x, y, topo[i_t-1], rstride=rstride, 
                                    cstride=cstride, 
                                    cmap=cmap,vmin=vmin, vmax=vmax,
                                    linewidth=0, antialiased=False)
    ax2_3d_surf.set_zlim(vmin, vmax)
    ax2_3d_surf.view_init(65, -60)
    title_str = 't = %s s' % str(t)
    plt.title(title_str)
    ax2_3d_surf, _ = mpl.colorbar.make_axes(plt.gca())
    cbar = mpl.colorbar.ColorbarBase(ax2_3d_surf, cmap=None,
                                     norm=mpl.colors.Normalize(vmin=vmin, vmax=vmax),
                                     label=' etch depth [um]')
    ax2_3d_surf.autoscale()    
    out_fig = 'img/' + \
    str(t) + '.png'
    plt.savefig(out_fig, bbox_inches='tight')
    plt.close()

fig3, ax3 = plt.subplots(figsize=(8,7))
for i,_ in enumerate(x):
    line_x = np.sqrt(x[i,i]**2 + y[i,i]**2)
    ax3.scatter(line_x,topo[i_t-1][i,i])
    ax3.autoscale()
plt.show()
fig3, ax3 = plt.subplots(figsize=(8,7))              
dummy_i = int(n_points/2)
plt.plot(x[dummy_i,:],topo[i_t-1][dummy_i,:])
plt.show()

The code provided simulates a dry etching process by modeling the physical and chemical interactions that occur during etching. The etching process is an important step in microfabrication, where it is used to create intricate patterns and structures on a surface. By simulating the etching process, researchers can optimize their etching techniques and achieve higher precision and accuracy in their microfabrication processes.

The simulation model used includes several key parameters such as the etch rate, selectivity, and ion energy. The etch rate is the speed at which material is removed from the surface, while selectivity refers to the ability to etch one material while leaving another intact. The ion energy parameter determines the amount of energy transferred to the surface during the etching process.

The code could be improved in several ways. First, it could include more complex models of the etching process, including a more detailed simulation of the plasma and the interactions between the ions and the surface. Second, it could incorporate more experimental data to better calibrate the model and ensure that it accurately represents the etching process in the real world. Third, the code could be optimized for performance to enable researchers to simulate larger and more complex etching processes.


Now, precribing custom Bosch, isotropic, and tapered (combined bosch and isotropic) etching steps can be modifyied if necessary in the following:

<h1> This part is not yet implemented (Bosch process) </h1>

In [ ]:
for i_bosch in range(recipe_steps[step]['bosch']):
                    if len(str(i_bosch)) < 3:
                        if len(str(i_bosch)) == 1: 
                            if i_bosch == 9:
                                i_bosch_str = '0' + str(i_bosch+1)
                            else:
                                i_bosch_str = '00' + str(i_bosch+1)
                        elif len(str(i_bosch)) == 2: 
                            if i_bosch == 99:
                                i_bosch_str = str(i_bosch+1)
                            else:
                                i_bosch_str = '0' + str(i_bosch+1)
                                
                    # initial bosch cycle key
                    key = step + '_bosch-iso' + i_cycle_str + \
                          '_bosch' + i_bosch_str + '_isotime0'
                    etch_grid[key] = []

The surface is evolved by using the computed normals of the surface and stepping points back along the normals by some user defined vertical and horizontal etch rates.


<img align="center" src="img/DESi.png" width="750" />
<h3 align="center">From the class, remember the effect of Deep dry etching of poly-Si</h3>


In [ ]:
    
C4F8 = 100  # sccm cubic centimeters per minute  
SF6 = 300  # sccm
bias = 10  # volts
time = 600  # seconds
opening = 100  # um

plt.close('all')

# load mask
im_dir = '/ExampleMasks/'
im_file = 'fillet_square.png'
im_path = im_dir + im_file
curr_im = cv2.imread(im_path, cv2.IMREAD_ANYDEPTH)   
curr_im = cv2.GaussianBlur(curr_im,(3,3),0)


rgb_im = cv2.cvtColor(curr_im, cv2.COLOR_GRAY2RGB)
     
cont_im, conts, hier = cv2.findContours(curr_im, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)    
conts_im = cv2.drawContours(rgb_im, conts, -1, (0,255,0),3)

t_start = 0
t_end = 600  # seconds
t_step = 5
rot90_mat = np.array([[np.cos(np.pi/2), -np.sin(np.pi/2)],
                      [np.sin(np.pi/2), np.cos(np.pi/2)]])
vert_rate = 287/600  # um/s

horiz_rate = 77/600  # um/s
pixel_um_conv = 251/90.4672  # px/um
cmap = 'gnuplot'  # 'inferno' 'viridis'  # 'hot'
vmin = -290  # expected range of depth for color bar (min)
vmax = 0

Now try changing the parameters to approximate the etching results as follows:


<img align="center" src="img/DEtop.gif" width="750" />
<h3 align="center">Top view of Dry Etching with the fillet mask example. Time steps of 5s.</h3>


<img src="img/DEiso.gif" width="750" />
<h3 align="center">Isometric view of Dry Etching with the fillet mask example. Time steps of 5s.</h3>


From the class, remember the equipment diagram used for Deep dry etching.
<img src="img/DEeqp.png" width="500" />


